In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV
import scikitplot as skplt
from sklearn import metrics
from collections import OrderedDict
from sklearn.datasets import make_classification

In [52]:
l =    ['TCAATC', 'CCAATT', 'GTTGAC', 'TGAT', 'GGTT', 'GGTTGA', 'GTTGA',
       'TAGCGT', 'CGGTTA', 'TTGACG', 'CGATA', 'CGTCAA', 'CAATCG', 'CGTTGA',
       'GTCGAA', 'GTTG', 'CGGTAA', 'CATACG', 'AAAAAA', 'ACCAAT', 'TCAA',
       'TTGTCG', 'CGATAG', 'AAAAA', 'CAAC', 'ATACGC', 'CAATTG', 'CCAAT',
       'CGCAAT', 'GGTA', 'CAAT', 'TGTCGA', 'CGAT', 'CAATCT', 'GCGTAC', 'GGTTG',
       'TTGA', 'GTTGGT', 'TCCAAT', 'TTGCG', 'ACGGTT', 'CGATTA', 'ATCAAT',
       'GCAATC', 'ATCAAC', 'GTTGAT', 'GTCAAT', 'CGTCGA', 'ATTGG', 'CGTAGT',
       'ATAGCG', 'CCGATA', 'CAATC', 'CATCAA', 'TTCGAC', 'TTGAC', 'TTGCGC',
       'GCGTTA', 'CCGTTA', 'ATACGG', 'CGCGTT', 'CGATC', 'CGATAA', 'ATCGTA',
       'TCGA', 'TCGGTT', 'GATC', 'CGGTT']

In [53]:
c = [f for f in l if len(f) < 6]

In [54]:
b = [f for f in l if len(f) == 6]

In [55]:
b1 = [f for f in b if f[0] == 'A' or f[0] == 'T']
b2 = [f for f in b if f[0] == 'G' or f[0] == 'C']

In [59]:
df1 = pd.concat([pd.read_csv('../ppnnw5.csv')[['Gen', 'die', 'Fam'] + c], 
                 pd.read_csv('../ppnnw6.csv')[b]], axis = 1)

#df1 = df1.loc[df1['die'] == 'no']
df1['Gen'] = 1

In [24]:
df1['Fam'].value_counts()

caliciviridae          1762
picornaviridae          864
coronaviridae           690
flaviviridae            390
potyviridae             362
betaflexiviridae        222
rhabdoviridae           220
paramyxoviridae         214
tombusviridae           136
astroviridae            132
alphaflexiviridae       110
luteoviridae            106
closteroviridae          90
virgaviridae             86
togaviridae              84
tymoviridae              82
iflaviridae              76
narnaviridae             72
endornaviridae           60
arteriviridae            48
dicistroviridae          40
solemoviridae            38
secoviridae              36
filoviridae              32
hepeviridae              32
pneumoviridae            30
bromoviridae             30
mesoniviridae            26
leviviridae              24
bornaviridae             24
hypoviridae              24
marnaviridae             20
nodaviridae              20
tobaniviridae            16
hantaviridae             12
polycipiviridae     

In [60]:
df2 = pd.concat([pd.read_csv('../h1w5.csv')[['die'] + c], 
                 pd.read_csv('../h1w6p1.csv')[b1],
                 pd.read_csv('../h1w6p2.csv')[b2]], axis = 1)
df2 = df2.loc[df2['die'] == 'no']
df2['Gen'] = 0

In [62]:
temp = []
df2test = df2.sample(frac = 0.3)
df2train = df2[~df2.index.isin(df2test.index)]

for fam in set(df1['Fam'].to_list()):
    df1test = df1.loc[df1['Fam'] == fam]
    df1train = df1.loc[~(df1['Fam'] == fam)]
    df1train = df1train.loc[df1train['die'] == 'no']
    
    X_train = pd.concat([df1train.iloc[:,3:], df2train.iloc[:,1:-1]], axis = 0).reset_index(drop = True)
    y_train = pd.concat([df1train.iloc[:,0], df2train.iloc[:,-1]], axis = 0).reset_index(drop = True)

    X_test = pd.concat([df1test.iloc[:,3:], df2test.iloc[:,1:-1]], axis = 0).reset_index(drop = True)
    y_test = pd.concat([df1test.iloc[:,0], df2test.iloc[:,-1]], axis = 0).reset_index(drop = True)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy')
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    #cv_bin = cross_val_score(classifier, X, y, cv=cvf, scoring='f1')
    #cv_mac = cross_val_score(classifier, X, y, cv=cvf, scoring='f1_macro')
    f1bin = f1_score(y_test, y_pred, average='binary')
    f1mac = f1_score(y_test, y_pred, average='macro')
    
    #temp.append((fam, len(df1test), sum(y_pred[np.arange(len(y_pred))[y_test==y_pred]]), f1bin, f1mac))    
    temp.append((fam, len(df1test), cm[1,1], cm[0,1], cm[0,0], cm[1,0], f1bin, f1mac))

    print('Fam: %s ** total: %d ** correct/TP: %d ** FP: %d **  TN: %d ** FN: %d ** f1bin: %f ** f1mac: %f' 
          % (fam, len(df1test), cm[1,1], cm[0,1], cm[0,0], cm[1,0], f1bin, f1mac))

lofocv_68 = pd.DataFrame(temp, columns = ['Fam', 'Total', 'Correct/TP', 'FP', 'TN', 'FN', 'f1bin', 'f1mac']) 

Fam: kitaviridae ** total: 2 ** correct/TP: 2 ** FP: 5 **  TN: 4638 ** FN: 0 ** f1bin: 0.444444 ** f1mac: 0.721953
Fam: hepeviridae ** total: 32 ** correct/TP: 32 ** FP: 5 **  TN: 4638 ** FN: 0 ** f1bin: 0.927536 ** f1mac: 0.963499
Fam: caliciviridae ** total: 1762 ** correct/TP: 1576 ** FP: 5 **  TN: 4638 ** FN: 186 ** f1bin: 0.942866 ** f1mac: 0.961345
Fam: luteoviridae ** total: 106 ** correct/TP: 106 ** FP: 7 **  TN: 4636 ** FN: 0 ** f1bin: 0.968037 ** f1mac: 0.983641
Fam: mesoniviridae ** total: 26 ** correct/TP: 26 ** FP: 5 **  TN: 4638 ** FN: 0 ** f1bin: 0.912281 ** f1mac: 0.955871
Fam: tymoviridae ** total: 82 ** correct/TP: 73 ** FP: 6 **  TN: 4637 ** FN: 9 ** f1bin: 0.906832 ** f1mac: 0.952609
Fam: roniviridae ** total: 4 ** correct/TP: 4 ** FP: 7 **  TN: 4636 ** FN: 0 ** f1bin: 0.533333 ** f1mac: 0.766289
Fam: peribunyaviridae ** total: 4 ** correct/TP: 1 ** FP: 6 **  TN: 4637 ** FN: 3 ** f1bin: 0.181818 ** f1mac: 0.590424
Fam: narnaviridae ** total: 72 ** correct/TP: 62 ** 

ValueError: 7 columns passed, passed data had 8 columns

In [41]:
a = np.array([1,0,0,1])
b = np.array([0,0,0,1])

In [67]:
pd.DataFrame(temp, columns = ['Fam', 'Total', 'Correct/TP', 'FP', 'TN', 'FN', 'f1bin', 'f1mac']).to_csv('lofocv68', index = False) 